In [278]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 500)


In [279]:
# Define model path if you want to load a model
use_model = True
model_path = "data/model_86.pt"

# Define if you want to use CPU or GPU
use_cpu = False
gpu_available = torch.cuda.is_available()


if gpu_available and not use_cpu:
	device = torch.device("cuda:0")
	print(f"GPU Support Enabled, Using: {torch.cuda.get_device_name(0)}")  # Prints the name of the GPU
else:
	device = torch.device("cpu")
	print("No GPU available, using CPU")

GPU Support Enabled, Using: NVIDIA GeForce RTX 2070 SUPER


In [280]:
df = pd.read_csv('data/final_data.csv')

In [281]:
df.head()

,status,orbital_period,semi_major_axis,planet_radius,transit_duration,insolation_flux,equilibrium_temperature,kepler_magnitude,stellar_radius,stellar_effective_temperature,stellar_surface_gravity,stellar_mass,stellar_metallicity
0,1,0.571336,0.01312,1.820,1.79000,4293.0,2063.0,11.958,0.95,5496.0,4.42,0.92,0.060
1,1,2.180535,0.03558,16.197,2.92780,1860.0,1828.0,11.468,1.26,6360.0,4.34,1.26,0.137
2,1,3.595100,0.04700,1.950,4.33000,1037.0,1616.0,11.364,1.71,5430.0,3.99,1.05,0.200
3,1,1.673902,0.04220,3.450,1.60560,990.0,1094.0,10.408,1.19,6120.0,4.36,1.18,0.140
4,1,0.658524,0.01306,3.300,0.33336,879.5,1515.0,11.386,0.70,4285.0,4.58,0.69,-0.120


In [282]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7386 entries, 0 to 7385
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   status                         7386 non-null   int64  
 1   orbital_period                 7386 non-null   float64
 2   semi_major_axis                7386 non-null   float64
 3   planet_radius                  7386 non-null   float64
 4   transit_duration               7386 non-null   float64
 5   insolation_flux                7386 non-null   float64
 6   equilibrium_temperature        7386 non-null   float64
 7   kepler_magnitude               7386 non-null   float64
 8   stellar_radius                 7386 non-null   float64
 9   stellar_effective_temperature  7386 non-null   float64
 10  stellar_surface_gravity        7386 non-null   float64
 11  stellar_mass                   7386 non-null   float64
 12  stellar_metallicity            7386 non-null   f

In [283]:
df.describe()

,status,orbital_period,semi_major_axis,planet_radius,transit_duration,insolation_flux,equilibrium_temperature,kepler_magnitude,stellar_radius,stellar_effective_temperature,stellar_surface_gravity,stellar_mass,stellar_metallicity
count,7386.000000,7386.000000,7386.00000,7386.000000,7386.000000,7.386000e+03,7386.000000,7386.000000,7386.000000,7386.000000,7386.000000,7386.000000,7386.000000
mean,0.382480,49.648224,0.18903,102.287896,5.708500,9.209798e+03,1151.313837,14.236909,1.720254,5707.695911,4.313374,1.021002,-0.126083
std,0.486026,117.546780,0.29365,3166.460197,6.833667,1.777975e+05,895.020440,1.413719,5.900127,822.079435,0.433957,0.351371,0.282097
min,0.000000,0.241843,0.00590,0.080000,0.104600,2.000000e-02,92.000000,6.966000,0.116000,2661.000000,0.047000,0.094000,-2.500000
25%,0.000000,2.237506,0.03340,1.510000,2.486083,2.714250e+01,582.000000,13.405000,0.827000,5301.500000,4.220000,0.840000,-0.260000
50%,0.000000,7.739694,0.07510,2.620000,3.842320,1.783800e+02,932.000000,14.495000,0.997000,5767.000000,4.438000,0.972000,-0.100000
75%,1.000000,27.392805,0.17610,23.802500,6.238627,1.122505e+03,1476.000000,15.303500,1.342750,6113.000000,4.544000,1.099000,0.070000
max,1.000000,3650.000000,4.50000,200346.000000,138.540000,1.094755e+07,14667.000000,20.003000,229.908000,15896.000000,5.283000,3.686000,0.560000


In [284]:
# Nomalize data
df = (df - df.min()) / (df.max() - df.min())
df.describe()

,status,orbital_period,semi_major_axis,planet_radius,transit_duration,insolation_flux,equilibrium_temperature,kepler_magnitude,stellar_radius,stellar_effective_temperature,stellar_surface_gravity,stellar_mass,stellar_metallicity
count,7386.000000,7386.000000,7386.000000,7386.000000,7386.000000,7386.000000,7386.000000,7386.000000,7386.000000,7386.000000,7386.000000,7386.000000,7386.000000
mean,0.382480,0.013537,0.040749,0.000510,0.040480,0.000841,0.072680,0.557713,0.006981,0.230200,0.814815,0.258074,0.775790
std,0.486026,0.032207,0.065341,0.015805,0.049364,0.016241,0.061408,0.108439,0.025676,0.062114,0.082880,0.097820,0.092189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000547,0.006119,0.000007,0.017203,0.000002,0.033619,0.493902,0.003094,0.199509,0.796982,0.207684,0.732026
50%,0.000000,0.002054,0.015398,0.000013,0.027000,0.000016,0.057633,0.577510,0.003834,0.234681,0.838617,0.244432,0.784314
75%,1.000000,0.007439,0.037872,0.000118,0.044310,0.000103,0.094957,0.639526,0.005339,0.260824,0.858862,0.279788,0.839869
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [285]:
# Split into train and test
train_df, test_df = train_test_split(df, test_size=0.2)

In [286]:
# Prep training data
x_train = train_df.drop('status', axis=1)
y_train = train_df['status']

# Convert to numpy arrays
x_train = np.array(x_train,dtype=np.float32)
y_train = np.array(y_train,dtype=np.float32).reshape(-1,1)

# Convert to tensors
X_train = torch.tensor(x_train, dtype=torch.float32)
Y_train = torch.tensor(y_train, dtype=torch.float32)

print(f"Input dimensions: {x_train.shape[1]}")
print(f"Output dimensions: {y_train.shape[1]}")

Input dimensions: 12
Output dimensions: 1


In [287]:
# Prep testing data
x_test = test_df.drop('status', axis=1)
y_test = test_df['status']

# Convert to numpy arrays
x_test = np.array(x_test,dtype=np.float32)
y_test = np.array(y_test,dtype=np.float32).reshape(-1,1)

# Convert to tensors
X_test = torch.tensor(x_test, dtype=torch.float32)
Y_test = torch.tensor(y_test, dtype=torch.float32)

In [288]:
# Creating/Loading the model
if use_model:
    model = torch.load(model_path)
else:
    model = nn.Sequential(
        nn.Linear(12, 24),
        nn.ReLU(),
        nn.Linear(24, 1),
        nn.ReLU(),
        nn.Linear(1, 1),
        nn.Sigmoid()
    )
print(model)

Sequential(
  (0): Linear(in_features=12, out_features=24, bias=True)
  (1): ReLU()
  (2): Linear(in_features=24, out_features=1, bias=True)
  (3): ReLU()
  (4): Linear(in_features=1, out_features=1, bias=True)
  (5): Sigmoid()
)


In [289]:
# Define loss function and optimizer
loss_fn = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [290]:
# Transfer model to GPU if available
if gpu_available and not use_cpu:
	print(f"Transferring model to GPU: {torch.cuda.get_device_name(0)}")
	model = model.cuda()
	loss_fn = loss_fn.cuda()

Transferring model to GPU: NVIDIA GeForce RTX 2070 SUPER


In [291]:
# Define training parameters
n_epochs = 100
batch_size = 10

# Train the model
for epoch in range(n_epochs):
    for i in range(0, len(X_train), batch_size):
        # Get batch
        X_batch = X_train[i:i+batch_size]
        Y_batch = Y_train[i:i+batch_size]

        # Transfer to GPU if available
        if gpu_available and not use_cpu:
            X_batch = X_batch.cuda()
            Y_batch = Y_batch.cuda()

        # Feedforward
        Y_pred = model(X_batch)

        # Calculate loss
        loss = loss_fn(Y_pred, Y_batch)

        # Backpropagate error and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print loss after each epoch
    print(f'Finished epoch {epoch}, latest loss {loss}')

Finished epoch 0, latest loss 0.3449837267398834
Finished epoch 1, latest loss 0.3493909537792206
Finished epoch 2, latest loss 0.35280826687812805
Finished epoch 3, latest loss 0.35414937138557434
Finished epoch 4, latest loss 0.3540843725204468
Finished epoch 5, latest loss 0.3526693284511566
Finished epoch 6, latest loss 0.35271093249320984
Finished epoch 7, latest loss 0.35360199213027954
Finished epoch 8, latest loss 0.3544597923755646
Finished epoch 9, latest loss 0.3523108661174774
Finished epoch 10, latest loss 0.35334333777427673
Finished epoch 11, latest loss 0.35252124071121216
Finished epoch 12, latest loss 0.35415321588516235
Finished epoch 13, latest loss 0.3511732816696167
Finished epoch 14, latest loss 0.36481773853302
Finished epoch 15, latest loss 0.35536760091781616
Finished epoch 16, latest loss 0.3571920394897461
Finished epoch 17, latest loss 0.3647938370704651
Finished epoch 18, latest loss 0.36312931776046753
Finished epoch 19, latest loss 0.3600563406944275
Fin

In [292]:
# Make predictions
X_test = X_test.to(device)
with torch.no_grad():
    # Run the model on the test data and transfer the result to CPU
    Y_pred = model(X_test).to("cpu")

In [293]:
# Calculate the accuracy
accuracy = (Y_pred.round() == Y_test).float().mean()
print(f"Accuracy {accuracy}")

Accuracy 0.8856562972068787


In [294]:
# torch.save(model, "data/model_88.pt")